In [46]:
# Set directory, which will be scanned recursively (including subdirectories) to find all .wav files, which are then appended to a list with all .wav-File Paths

import os

#directory = "/Users/tillmann/Dropbox/mitralabs/recordings"
directory = "/Users/tillmann/Library/Mobile Documents/com~apple~CloudDocs/recordings/private"
wav_files = []

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".wav"):
            wav_files.append(os.path.join(root, file))


In [41]:
# Function to check the status of the orchestrator and return the status
import requests
def check_status():
    url = "http://localhost:3030/status"
    # set header X-API-KEY = "test"
    headers = {
        "X-API-KEY":
        "test"
    }   
    response = requests.get(url, headers=headers)

    return response.json()

check_status()

{'status': 'success',
 'active_tasks': 0,
 'max_tasks': 2,
 'available_capacity': 2}

In [42]:
def upload_audio_file(file_path, api_key="test", base_url="http://localhost:3030"):
    """
    Upload an audio file to the orchestrator service.
    
    Args:
        file_path (str): Path to the audio file to upload
        api_key (str): API key for authentication (default: "test")
        base_url (str): Base URL of the service (default: "http://localhost:8000")
        
    Returns:
        requests.Response: HTTP response from the server
    """
    # Extract filename from path
    filename = os.path.basename(file_path)
    
    # Set headers
    headers = {
        "X-API-Key": api_key,
        "Content-Disposition": f'attachment; filename="{filename}"'
    }
    
    # Read file content
    with open(file_path, "rb") as file:
        file_content = file.read()
    
    # Send request
    response = requests.post(
        f"{base_url}/uploadAudio",
        headers=headers,
        data=file_content
    )
    
    return response

In [47]:
# Sort Files by session id and index

def sort_files(wav_files):
    sorted_files = []
    for file in wav_files:
        try:
            file_name = file.split("/")[-1]
            session_id = file_name.split("_")[0]
            index = file_name.split("_")[1]
            sorted_files.append((session_id, index, file))
        except:
            pass

    # Sort files but account that index 2 comes after 1 and not after 19
    sorted_files.sort(key=lambda x: (x[0], int(x[1])))
    return sorted_files

In [57]:
# Upload all .wav files to the orchestrator but before sending an HTTP request, check the status of the orchestrator

import time

sorted_files = sort_files(wav_files)

# Filter for session id 374
sorted_files = [file for file in sorted_files if file[0] == "374"]

for i,file_path in enumerate(sorted_files):

    while(check_status()["available_capacity"] == 0):
        print("Orchestrator is busy. Please try again later.")
        # Wait for 1 second
        time.sleep(1)

    file_path = file_path[2]

    response = upload_audio_file(file_path)
    print(f"Uploaded file {i+1}/{len(wav_files)}: {file_path} - Status: {response.status_code}")

    if i == 10:
        break   


Uploaded file 1/2743: /Users/tillmann/Library/Mobile Documents/com~apple~CloudDocs/recordings/private/session_374/374_0_25-03-06_10-44-21_start.wav - Status: 200
Uploaded file 2/2743: /Users/tillmann/Library/Mobile Documents/com~apple~CloudDocs/recordings/private/session_374/374_1_25-03-06_10-44-31_middle.wav - Status: 200
Uploaded file 3/2743: /Users/tillmann/Library/Mobile Documents/com~apple~CloudDocs/recordings/private/session_374/374_2_25-03-06_10-44-41_middle.wav - Status: 200
Uploaded file 4/2743: /Users/tillmann/Library/Mobile Documents/com~apple~CloudDocs/recordings/private/session_374/374_3_25-03-06_10-44-51_middle.wav - Status: 200
Orchestrator is busy. Please try again later.
Orchestrator is busy. Please try again later.
Orchestrator is busy. Please try again later.
Orchestrator is busy. Please try again later.
Orchestrator is busy. Please try again later.
Orchestrator is busy. Please try again later.
Orchestrator is busy. Please try again later.
Orchestrator is busy. Pleas